# Busca de Hiperparâmetros com Algoritmo Genético para LSTM (PyTorch)
Este notebook aplica um algoritmo genético simples para encontrar os melhores hiperparâmetros de uma rede LSTM que prevê o número mensal de crimes em Toronto.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import random, warnings
warnings.filterwarnings('ignore')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
df = pd.read_csv('C:/Users/peder/Downloads/major-crime-indicators.csv', on_bad_lines='skip', parse_dates=['OCC_DATE'])
df = df[df['OCC_DATE'].notna()].sort_values('OCC_DATE')
ts = df.set_index('OCC_DATE').resample('M').size().to_frame('count')
ts = ts[ts.index >= '2014-01-01']
scaler = MinMaxScaler()
ts['scaled'] = scaler.fit_transform(ts[['count']])
data = ts['scaled'].values
print(ts.head())

            count    scaled
OCC_DATE                   
2014-01-31   2601  0.202855
2014-02-28   2262  0.056228
2014-03-31   2508  0.162630
2014-04-30   2576  0.192042
2014-05-31   2914  0.338235


In [11]:
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [12]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        return self.linear(out).squeeze()

In [13]:
def evaluate_model(hidden_size, dropout, lr, batch_size, epochs, window_size):
    X, y = create_sequences(data, window_size)
    if len(X) == 0: return float('inf')
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1)
    y_val = torch.tensor(y_val, dtype=torch.float32)
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)
    model = LSTMModel(1, hidden_size, dropout).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for _ in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            output = model(X_batch.to(device))
            predictions.extend(output.cpu().numpy())
            actuals.extend(y_batch.numpy())
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    return rmse

In [14]:
# Algoritmo Genético para busca
population_size = 8
generations = 5
param_space = {
    'hidden_size': [32, 64, 128],
    'dropout': [0.1, 0.2, 0.3],
    'lr': [0.001, 0.005],
    'batch_size': [8, 16],
    'epochs': [5, 10, 15, 50, 100, 250 , 500, 1000],
    'window_size': [6, 12]
}

def random_individual():
    return {k: random.choice(v) for k, v in param_space.items()}

population = [random_individual() for _ in range(population_size)]
for gen in range(generations):
    print(f"\n🧬 Geração {gen+1}")
    scored = []
    for ind in population:
        score = evaluate_model(**ind)
        scored.append((score, ind))
        print(f"{ind} -> RMSE: {score:.4f}")
    scored.sort()
    population = [scored[i][1] for i in range(population_size//2)]
    while len(population) < population_size:
        parent = random.choice(population)
        child = parent.copy()
        key = random.choice(list(param_space.keys()))
        child[key] = random.choice(param_space[key])
        population.append(child)
print(f"\nMelhor configuração encontrada: {scored[0][1]} com RMSE={scored[0][0]:.4f}")


🧬 Geração 1
{'hidden_size': 32, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 8, 'epochs': 100, 'window_size': 6} -> RMSE: 0.1028
{'hidden_size': 32, 'dropout': 0.1, 'lr': 0.005, 'batch_size': 8, 'epochs': 50, 'window_size': 12} -> RMSE: 0.0992
{'hidden_size': 64, 'dropout': 0.3, 'lr': 0.005, 'batch_size': 8, 'epochs': 10, 'window_size': 6} -> RMSE: 0.1326
{'hidden_size': 128, 'dropout': 0.2, 'lr': 0.001, 'batch_size': 16, 'epochs': 500, 'window_size': 6} -> RMSE: 0.2168
{'hidden_size': 128, 'dropout': 0.2, 'lr': 0.005, 'batch_size': 16, 'epochs': 1000, 'window_size': 12} -> RMSE: 0.1169
{'hidden_size': 128, 'dropout': 0.3, 'lr': 0.005, 'batch_size': 8, 'epochs': 250, 'window_size': 12} -> RMSE: 0.1730
{'hidden_size': 64, 'dropout': 0.1, 'lr': 0.005, 'batch_size': 8, 'epochs': 500, 'window_size': 6} -> RMSE: 0.1692
{'hidden_size': 64, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 8, 'epochs': 1000, 'window_size': 6} -> RMSE: 0.1824

🧬 Geração 2
{'hidden_size': 32, 'dropout': 0.1, 'lr': 0.00